In [46]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving no_early_dates_90_days.csv to no_early_dates_90_days (2).csv


In [47]:
import pandas as pd


df = pd.read_csv('no_early_dates_90_days.csv')





In [48]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'channelDescription', 'channelJoinedDate', 'channelTotalVideos',
       'channelTotalViews', 'channelUsername', 'commentsCount', 'date',
       'duration', 'id', 'isChannelVerified', 'likes', 'numberOfSubscribers',
       'order', 'text', 'title', 'url', 'viewCount', 'likes_per_subscriber',
       'comments_per_subscriber', 'views_per_subscriber',
       'duration_in_seconds', 'datetime_date', 'hashtags', 'comm_to_views',
       'likes_to_views', 'popular_brand', 'has_title_affiliate',
       'has_description_affiliate', 'has_channel_description_affiliate',
       'has_any_affiliate', 'has_business_inquiry',
       'engagement_per_subscriber', 'product', 'budget', 'self_ref', 'acronym',
       'korean', 'speed', 'skills/teach', 'skincare', 'comparing_products',
       'datetime', 'hour', 'day_of_week', 'month', 'year', 'day_name',
       'engagement_rate', 'prime_time', 'cluster1', 'prime_time1',
       'postin

In [49]:
all_features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref",
                 "acronym", "korean", "speed", "skills", "skincare", "comparing_products",
                 "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]


In [50]:
df["engagement"] = (df["likes"] + df["commentsCount"])  / (df["viewCount"] + 1)

df["normalized_view_conversion"] = np.log(df["viewCount"]/df["numberOfSubscribers"])

In [51]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score


all_features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref",
                 "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products",
                 "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

target1 = "engagement"
target2='normalized_view_conversion'


X = df[all_features]
y1 = df[target1]
y2 = df[target2]
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Running LassoCV to find optimal alpha...")
lasso_cv = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=np.logspace(-4, 1, 30))
lasso_cv.fit(X_train_scaled, y_train)
best_alpha = lasso_cv.alpha_

print(f"Best alpha found by CV: {best_alpha:.6f}")

# Evaluate LassoCV model
y_pred_cv = lasso_cv.predict(X_test_scaled)
mse_cv = mean_squared_error(y_test, y_pred_cv)
rmse_cv = np.sqrt(mse_cv)
mae_cv = mean_absolute_error(y_test, y_pred_cv)
r2_cv = r2_score(y_test, y_pred_cv)
exp_var_cv = explained_variance_score(y_test, y_pred_cv)

print("\nLassoCV Model Performance:")
print(f"Mean Squared Error (MSE): {mse_cv:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse_cv:.6f}")
print(f"Mean Absolute Error (MAE): {mae_cv:.6f}")
print(f"R² Score: {r2_cv:.6f}")
print(f"Explained Variance Score: {exp_var_cv:.6f}")

# feature importance for LassoCV
cv_coef = pd.DataFrame({'Feature': all_features, 'Coefficient': lasso_cv.coef_})
print("\nFeatures sorted by importance (LassoCV):")
print(cv_coef.sort_values('Coefficient', key=abs, ascending=False))

# Features selected by LassoCV (non-zero coefficients)
selected_features_cv = cv_coef[cv_coef['Coefficient'] != 0]['Feature'].tolist()
print(f"\nSelected {len(selected_features_cv)} features: {selected_features_cv}")


Running LassoCV to find optimal alpha...
Best alpha found by CV: 0.000329

LassoCV Model Performance:
Mean Squared Error (MSE): 0.001147
Root Mean Squared Error (RMSE): 0.033867
Mean Absolute Error (MAE): 0.021897
R² Score: 0.039337
Explained Variance Score: 0.040354

Features sorted by importance (LassoCV):
               Feature  Coefficient
0        popular_brand    -0.003154
9             skincare    -0.002977
11          prime_hour     0.001818
3               budget    -0.001584
7                speed    -0.001470
10  comparing_products    -0.001418
2              product    -0.001302
8         skills/teach    -0.001175
4             self_ref     0.001032
6               korean     0.001011
1    has_any_affiliate     0.000942
12            hashtags     0.000190
14         hasAdinText    -0.000101
5              acronym    -0.000000
13        hasAdinTitle     0.000000

Selected 13 features: ['popular_brand', 'has_any_affiliate', 'product', 'budget', 'self_ref', 'korean', 'speed', 

In [52]:

X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled2 = scaler.fit_transform(X_train)
X_test_scaled2 = scaler.transform(X_test)


print("Running LassoCV to find optimal alpha...")
lasso_cv2 = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=np.logspace(-4, 1, 30))
lasso_cv2.fit(X_train_scaled2, y_train2)
best_alpha2 = lasso_cv2.alpha_

print(f"Best alpha found by CV: {best_alpha:.6f}")

# Evaluate LassoCV model
y_pred_cv2 = lasso_cv2.predict(X_test_scaled2)
mse_cv = mean_squared_error(y_test2, y_pred_cv2)
rmse_cv = np.sqrt(mse_cv)
mae_cv = mean_absolute_error(y_test2, y_pred_cv2)
r2_cv = r2_score(y_test2, y_pred_cv2)
exp_var_cv = explained_variance_score(y_test2, y_pred_cv2)

print("\nLassoCV Model Performance:")
print(f"Mean Squared Error (MSE): {mse_cv:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse_cv:.6f}")
print(f"Mean Absolute Error (MAE): {mae_cv:.6f}")
print(f"R² Score: {r2_cv:.6f}")
print(f"Explained Variance Score: {exp_var_cv:.6f}")

# feature importance for LassoCV
cv_coef2 = pd.DataFrame({'Feature': all_features, 'Coefficient': lasso_cv2.coef_})
print("\nFeatures sorted by importance (LassoCV):")
print(cv_coef2.sort_values('Coefficient', key=abs, ascending=False))

# Features selected by LassoCV (non-zero coefficients)
selected_features_cv2 = cv_coef2[cv_coef2['Coefficient'] != 0]['Feature'].tolist()
print(f"\nSelected {len(selected_features_cv2)} features: {selected_features_cv2}")


Running LassoCV to find optimal alpha...
Best alpha found by CV: 0.000329

LassoCV Model Performance:
Mean Squared Error (MSE): 2.197166
Root Mean Squared Error (RMSE): 1.482284
Mean Absolute Error (MAE): 1.150399
R² Score: 0.108848
Explained Variance Score: 0.111581

Features sorted by importance (LassoCV):
               Feature  Coefficient
6               korean     0.282699
1    has_any_affiliate    -0.259581
5              acronym     0.157295
2              product     0.148926
3               budget     0.115604
14         hasAdinText    -0.115111
10  comparing_products     0.095308
7                speed     0.074421
11          prime_hour     0.068833
0        popular_brand     0.048740
9             skincare    -0.037227
8         skills/teach     0.034482
13        hasAdinTitle     0.022308
4             self_ref     0.005374
12            hashtags     0.001479

Selected 15 features: ['popular_brand', 'has_any_affiliate', 'product', 'budget', 'self_ref', 'acronym', 'korean'

In [53]:
all_features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref",
                 "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products",
                 "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

target1 = "engagement"
target2='normalized_view_conversion'


X = df[all_features]
y1 = df[target1]
y2 = df[target2]
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print("\nEvaluating different alpha values:")
alphas = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 10.0]
results = []

for alpha in alphas:

    lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)


    cv_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=5,
                                scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())

    lasso.fit(X_train_scaled, y_train)

    y_pred = lasso.predict(X_test_scaled)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    exp_var = explained_variance_score(y_test, y_pred)

    # Count non-zero coefficients
    n_features = np.sum(lasso.coef_ != 0)

    results.append({
        'alpha': alpha,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'exp_var': exp_var,
        'cv_rmse': cv_rmse,
        'n_features': n_features,
        'model': lasso
    })

    print(f"\nAlpha: {alpha}")
    print(f"Test MSE: {mse:.6f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}")
    print(f"R² Score: {r2:.6f}, Explained Variance: {exp_var:.6f}")
    print(f"Cross-validated RMSE: {cv_rmse:.6f}")
    print(f"Number of features selected: {n_features}")

    # Top 5 features by importance
    coef_df = pd.DataFrame({'Feature': all_features, 'Coefficient': lasso.coef_})
    print("Top 5 features by importance:")
    print(coef_df.sort_values('Coefficient', key=abs, ascending=False).head(5))

# Find best model based on  R² score
results_df = pd.DataFrame([{
    'alpha': r['alpha'],
    'mse': r['mse'],
    'rmse': r['rmse'],
    'mae': r['mae'],
    'r2': r['r2'],
    'exp_var': r['exp_var'],
    'cv_rmse': r['cv_rmse'],
    'n_features': r['n_features']
} for r in results])

print("\nSummary of all alpha values tested:")
print(results_df)

#  best alpha based on R² score
best_idx = results_df['r2'].idxmax()
best_alpha_manual = results_df.loc[best_idx, 'alpha']
print(f"\nBest alpha from manual testing: {best_alpha_manual} (based on highest R²)")

final_model = results[best_idx]['model']
final_coef = pd.DataFrame({'Feature': all_features, 'Coefficient': final_model.coef_})
print("\nFeatures sorted by importance (best manual model):")
print(final_coef.sort_values('Coefficient', key=abs, ascending=False))

# Features selected by best model
selected_features = final_coef[final_coef['Coefficient'] != 0]['Feature'].tolist()
print(f"\nSelected {len(selected_features)} features: {selected_features}")




Evaluating different alpha values:

Alpha: 0.0001
Test MSE: 0.001146, RMSE: 0.033847, MAE: 0.021927
R² Score: 0.040487, Explained Variance: 0.041518
Cross-validated RMSE: 0.032247
Number of features selected: 15
Top 5 features by importance:
          Feature  Coefficient
0   popular_brand    -0.003376
9        skincare    -0.003313
11     prime_hour     0.002048
3          budget    -0.001911
7           speed    -0.001823

Alpha: 0.001
Test MSE: 0.001157, RMSE: 0.034020, MAE: 0.022074
R² Score: 0.030643, Explained Variance: 0.031654
Cross-validated RMSE: 0.032302
Number of features selected: 9
Top 5 features by importance:
               Feature  Coefficient
0        popular_brand    -0.002451
9             skincare    -0.002095
11          prime_hour     0.001168
3               budget    -0.000770
10  comparing_products    -0.000727

Alpha: 0.01
Test MSE: 0.001195, RMSE: 0.034566, MAE: 0.022750
R² Score: -0.000695, Explained Variance: 0.000000
Cross-validated RMSE: 0.032689
Number

In [54]:
all_features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref",
                 "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products",
                 "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

target1 = "engagement"
target2='normalized_view_conversion'


X = df[all_features]
y1 = df[target1]
y2 = df[target2]
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print("\nEvaluating different alpha values:")
alphas = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 10.0]
results = []

for alpha in alphas:

    lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)


    cv_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=5,
                                scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())

    lasso.fit(X_train_scaled, y_train)

    y_pred = lasso.predict(X_test_scaled)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    exp_var = explained_variance_score(y_test, y_pred)

    # Count non-zero coefficients
    n_features = np.sum(lasso.coef_ != 0)

    results.append({
        'alpha': alpha,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'exp_var': exp_var,
        'cv_rmse': cv_rmse,
        'n_features': n_features,
        'model': lasso
    })

    print(f"\nAlpha: {alpha}")
    print(f"Test MSE: {mse:.6f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}")
    print(f"R² Score: {r2:.6f}, Explained Variance: {exp_var:.6f}")
    print(f"Cross-validated RMSE: {cv_rmse:.6f}")
    print(f"Number of features selected: {n_features}")

    # Top 5 features by importance
    coef_df = pd.DataFrame({'Feature': all_features, 'Coefficient': lasso.coef_})
    print("Top 5 features by importance:")
    print(coef_df.sort_values('Coefficient', key=abs, ascending=False).head(5))

# Find best model based on  R² score
results_df = pd.DataFrame([{
    'alpha': r['alpha'],
    'mse': r['mse'],
    'rmse': r['rmse'],
    'mae': r['mae'],
    'r2': r['r2'],
    'exp_var': r['exp_var'],
    'cv_rmse': r['cv_rmse'],
    'n_features': r['n_features']
} for r in results])

print("\nSummary of all alpha values tested:")
print(results_df)

#  best alpha based on R² score
best_idx = results_df['r2'].idxmax()
best_alpha_manual = results_df.loc[best_idx, 'alpha']
print(f"\nBest alpha from manual testing: {best_alpha_manual} (based on highest R²)")

final_model = results[best_idx]['model']
final_coef = pd.DataFrame({'Feature': all_features, 'Coefficient': final_model.coef_})
print("\nFeatures sorted by importance (best manual model):")
print(final_coef.sort_values('Coefficient', key=abs, ascending=False))

# Features selected by best model
selected_features = final_coef[final_coef['Coefficient'] != 0]['Feature'].tolist()
print(f"\nSelected {len(selected_features)} features: {selected_features}")




Evaluating different alpha values:

Alpha: 0.0001
Test MSE: 2.196274, RMSE: 1.481983, MAE: 1.149909
R² Score: 0.109210, Explained Variance: 0.111925
Cross-validated RMSE: 1.545699
Number of features selected: 15
Top 5 features by importance:
              Feature  Coefficient
6              korean     0.287273
1   has_any_affiliate    -0.259484
5             acronym     0.160791
2             product     0.151366
14        hasAdinText    -0.122273

Alpha: 0.001
Test MSE: 2.196470, RMSE: 1.482049, MAE: 1.150025
R² Score: 0.109130, Explained Variance: 0.111850
Cross-validated RMSE: 1.545666
Number of features selected: 15
Top 5 features by importance:
              Feature  Coefficient
6              korean     0.286084
1   has_any_affiliate    -0.259509
5             acronym     0.159882
2             product     0.150731
14        hasAdinText    -0.120411

Alpha: 0.01
Test MSE: 2.198941, RMSE: 1.482883, MAE: 1.151588
R² Score: 0.108128, Explained Variance: 0.110891
Cross-validated RMS